### Load the important libraries

In [117]:
pip install thefuzz

  Obtaining dependency information for thefuzz from https://files.pythonhosted.org/packages/82/4f/1695e70ceb3604f19eda9908e289c687ea81c4fecef4d90a9d1d0f2f7ae9/thefuzz-0.22.1-py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.


In [184]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from thefuzz import fuzz, process
import swifter
sns.set()

### We start with loading data from merged_data from interim data from the last step

In [6]:
#Import data saved from the last saved data
merged_data = pd.read_csv("../data/interim/merged_data.csv")

In [7]:
merged_data.head(5)

,Title,description,authors,categories,User_id,review/score,review/summary,review/text
0,Its Only Art If Its Well Hung!,NaN,['Julie Strain'],['Comics & Graphic Novels'],AVCGYZL8FQQTD,4.0,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],['Biography & Autobiography'],A30TK6U7DNS82R,5.0,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],['Biography & Autobiography'],A3UH4UZ4RSVO82,5.0,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],['Biography & Autobiography'],A2MVUWT453QH61,4.0,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],['Biography & Autobiography'],A22X4XUPKF66MR,4.0,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...


In [32]:
book_category_data = pd.read_csv("../data/raw/goodreads_data.csv", index_col=0)

In [33]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1

In [162]:
merged_data_short = merged_data.sample(15000)

In [163]:
unique_title_df = pd.DataFrame({'titles':merged_data_short.Title.unique()})
matches = unique_title_df['titles'].swifter.apply(lambda x: process.extractOne(x, book_category_data.Book))

Pandas Apply:   0%|          | 0/9614 [00:00<?, ?it/s]

In [178]:
book_category_data.head()

,Book,Author,Description,Genres,Avg_Rating,Num_Ratings,URL
0,To Kill a Mockingbird,Harper Lee,The unforgettable novel of a childhood in a sl...,"['Classics', 'Fiction', 'Historical Fiction', ...",4.27,"5,691,311",https://www.goodreads.com/book/show/2657.To_Ki...
1,Harry Potter and the Philosopher’s Stone (Harr...,J.K. Rowling,Harry Potter thinks he is an ordinary boy - un...,"['Fantasy', 'Fiction', 'Young Adult', 'Magic',...",4.47,"9,278,135",https://www.goodreads.com/book/show/72193.Harr...
2,Pride and Prejudice,Jane Austen,"Since its immediate success in 1813, Pride and...","['Classics', 'Fiction', 'Romance', 'Historical...",4.28,"3,944,155",https://www.goodreads.com/book/show/1885.Pride...
3,The Diary of a Young Girl,Anne Frank,Discovered in the attic in which she spent the...,"['Classics', 'Nonfiction', 'History', 'Biograp...",4.18,"3,488,438",https://www.goodreads.com/book/show/48855.The_...
4,Animal Farm,George Orwell,Librarian's note: There is an Alternate Cover ...,"['Classics', 'Fiction', 'Dystopia', 'Fantasy',...",3.98,"3,575,172",https://www.goodreads.com/book/show/170448.Ani...


In [170]:
book_category_matches = pd.DataFrame({})

pandas.core.series.Series

In [179]:
book_genres_merged = matches.apply(lambda x: [x[0], x[1], book_category_data.iloc[x[2]].Genres])

In [183]:
book_genres_merged[0]

['To Kill a Mockingbird',
 86,
 "['Classics', 'Fiction', 'Historical Fiction', 'School', 'Literature', 'Young Adult', 'Historical']"]

In [94]:
genresList = []
for x in book_category_data.Genres.str.strip("[]").str.split(","):
    print (x[0])

'Classics'
'Fantasy'
'Classics'
'Classics'
'Classics'
'Classics'
'Classics'
'Classics'
'Classics'
'Fantasy'
'Historical Fiction'
'Classics'
'Fantasy'
'Classics'
'Classics'
'Fantasy'
'Fiction'
'Young Adult'
'Childrens'
'Classics'
'Classics'
'Young Adult'
'Fantasy'
'Classics'
'Classics'
'Childrens'
'Fantasy'
'Classics'
'Classics'
'Classics'
'Nonfiction'
'Classics'
'Religion'
'Fantasy'
'Science Fiction'
'Classics'
'Fantasy'
'Classics'
'Fantasy'
'Classics'
'Fiction'
'Classics'
'Classics'
'Fiction'
'Classics'
'Fiction'
'Fantasy'
'Classics'
'Classics'
'Childrens'
'Classics'
'Fiction'
'Classics'
'Fiction'
'Classics'
'Poetry'
'Classics'
'Science Fiction'
'Childrens'
'Classics'
'Classics'
'Classics'
'Fiction'
'Fiction'
'Classics'
'Classics'
'Fiction'
'Childrens'
'Classics'
'Nonfiction'
'Classics'
'Fantasy'
'Classics'
'Classics'
'Classics'
'Classics'
'Classics'
'Fantasy'
'Classics'
'Classics'
'Fiction'
'Classics'
'Classics'
'Childrens'
'Fantasy'
'Classics'
'Classics'
'Classics'
'Classics'
'Class

In [34]:
book_category_data.head(5)

,Book,Author,Description,Genres,Avg_Rating,Num_Ratings,URL
0,To Kill a Mockingbird,Harper Lee,The unforgettable novel of a childhood in a sl...,"['Classics', 'Fiction', 'Historical Fiction', ...",4.27,"5,691,311",https://www.goodreads.com/book/show/2657.To_Ki...
1,Harry Potter and the Philosopher’s Stone (Harr...,J.K. Rowling,Harry Potter thinks he is an ordinary boy - un...,"['Fantasy', 'Fiction', 'Young Adult', 'Magic',...",4.47,"9,278,135",https://www.goodreads.com/book/show/72193.Harr...
2,Pride and Prejudice,Jane Austen,"Since its immediate success in 1813, Pride and...","['Classics', 'Fiction', 'Romance', 'Historical...",4.28,"3,944,155",https://www.goodreads.com/book/show/1885.Pride...
3,The Diary of a Young Girl,Anne Frank,Discovered in the attic in which she spent the...,"['Classics', 'Nonfiction', 'History', 'Biograp...",4.18,"3,488,438",https://www.goodreads.com/book/show/48855.The_...
4,Animal Farm,George Orwell,Librarian's note: There is an Alternate Cover ...,"['Classics', 'Fiction', 'Dystopia', 'Fantasy',...",3.98,"3,575,172",https://www.goodreads.com/book/show/170448.Ani...


In [95]:
selected_titles = merged_data.Title.unique()
books_categories_merged = pd.DataFrame(columns = ['Title', 'Book', 'Ratio_Value', 'Categories', 'More_Than_one_merge'])
duplicate = 0
for x in selected_titles:
    duplicate = 0
    for index,row in book_category_data.iterrows():
        #print (row['Book'])
        fuzz_ratio_value = fuzz.ratio(row['Book'], x)
        if fuzz_ratio_value > 80:
            print (fuzz_ratio_value)
            print (books_categories_merged.head())
            books_categories_merged.loc[len(books_categories_merged)] = [x, row['Book'], fuzz_ratio_value, row['Genres'], duplicate]

books_categories_merged.to_csv("../data/interim/merged_data_genres.csv", index=False)


91
Empty DataFrame
Columns: [Title, Book, Ratio_Value, Categories, More_Than_one_merge]
Index: []
100
                     Title                     Book  Ratio_Value  \
0  Economics in one lesson  Economics in One Lesson           91   

                                          Categories  More_Than_one_merge  
0  ['Economics', 'Nonfiction', 'Business', 'Polit...                    0  
86
                     Title                     Book  Ratio_Value  \
0  Economics in one lesson  Economics in One Lesson           91   
1                   Island                   Island          100   

                                          Categories  More_Than_one_merge  
0  ['Economics', 'Nonfiction', 'Business', 'Polit...                    0  
1  ['Fiction', 'Classics', 'Science Fiction', 'Ph...                    0  
91
                     Title                     Book  Ratio_Value  \
0  Economics in one lesson  Economics in One Lesson           91   
1                   Island        

100
                     Title                       Book  Ratio_Value  \
0  Economics in one lesson    Economics in One Lesson           91   
1                   Island                     Island          100   
2                   Island                   Islandia           86   
3    Man Who Planted Trees  The Man Who Planted Trees           91   
4       The Fire Next Time         The Fire Next Time          100   

                                          Categories  More_Than_one_merge  
0  ['Economics', 'Nonfiction', 'Business', 'Polit...                    0  
1  ['Fiction', 'Classics', 'Science Fiction', 'Ph...                    0  
2  ['Fantasy', 'Fiction', 'Science Fiction', 'Cla...                    0  
3  ['Fiction', 'Short Stories', 'Classics', 'Fran...                    0  
4  ['Nonfiction', 'Essays', 'Classics', 'Race', '...                    0  
100
                     Title                       Book  Ratio_Value  \
0  Economics in one lesson    Economics in On

82
                     Title                       Book  Ratio_Value  \
0  Economics in one lesson    Economics in One Lesson           91   
1                   Island                     Island          100   
2                   Island                   Islandia           86   
3    Man Who Planted Trees  The Man Who Planted Trees           91   
4       The Fire Next Time         The Fire Next Time          100   

                                          Categories  More_Than_one_merge  
0  ['Economics', 'Nonfiction', 'Business', 'Polit...                    0  
1  ['Fiction', 'Classics', 'Science Fiction', 'Ph...                    0  
2  ['Fantasy', 'Fiction', 'Science Fiction', 'Cla...                    0  
3  ['Fiction', 'Short Stories', 'Classics', 'Fran...                    0  
4  ['Nonfiction', 'Essays', 'Classics', 'Race', '...                    0  
100
                     Title                       Book  Ratio_Value  \
0  Economics in one lesson    Economics in One

83
                     Title                       Book  Ratio_Value  \
0  Economics in one lesson    Economics in One Lesson           91   
1                   Island                     Island          100   
2                   Island                   Islandia           86   
3    Man Who Planted Trees  The Man Who Planted Trees           91   
4       The Fire Next Time         The Fire Next Time          100   

                                          Categories  More_Than_one_merge  
0  ['Economics', 'Nonfiction', 'Business', 'Polit...                    0  
1  ['Fiction', 'Classics', 'Science Fiction', 'Ph...                    0  
2  ['Fantasy', 'Fiction', 'Science Fiction', 'Cla...                    0  
3  ['Fiction', 'Short Stories', 'Classics', 'Fran...                    0  
4  ['Nonfiction', 'Essays', 'Classics', 'Race', '...                    0  
83
                     Title                       Book  Ratio_Value  \
0  Economics in one lesson    Economics in One 

100
                     Title                       Book  Ratio_Value  \
0  Economics in one lesson    Economics in One Lesson           91   
1                   Island                     Island          100   
2                   Island                   Islandia           86   
3    Man Who Planted Trees  The Man Who Planted Trees           91   
4       The Fire Next Time         The Fire Next Time          100   

                                          Categories  More_Than_one_merge  
0  ['Economics', 'Nonfiction', 'Business', 'Polit...                    0  
1  ['Fiction', 'Classics', 'Science Fiction', 'Ph...                    0  
2  ['Fantasy', 'Fiction', 'Science Fiction', 'Cla...                    0  
3  ['Fiction', 'Short Stories', 'Classics', 'Fran...                    0  
4  ['Nonfiction', 'Essays', 'Classics', 'Race', '...                    0  
100
                     Title                       Book  Ratio_Value  \
0  Economics in one lesson    Economics in On

...

100
                     Title                       Book  Ratio_Value  \
0  Economics in one lesson    Economics in One Lesson           91   
1                   Island                     Island          100   
2                   Island                   Islandia           86   
3    Man Who Planted Trees  The Man Who Planted Trees           91   
4       The Fire Next Time         The Fire Next Time          100   

                                          Categories  More_Than_one_merge  
0  ['Economics', 'Nonfiction', 'Business', 'Polit...                    0  
1  ['Fiction', 'Classics', 'Science Fiction', 'Ph...                    0  
2  ['Fantasy', 'Fiction', 'Science Fiction', 'Cla...                    0  
3  ['Fiction', 'Short Stories', 'Classics', 'Fran...                    0  
4  ['Nonfiction', 'Essays', 'Classics', 'Race', '...                    0  
82
                     Title                       Book  Ratio_Value  \
0  Economics in one lesson    Economics in One

81
                     Title                       Book  Ratio_Value  \
0  Economics in one lesson    Economics in One Lesson           91   
1                   Island                     Island          100   
2                   Island                   Islandia           86   
3    Man Who Planted Trees  The Man Who Planted Trees           91   
4       The Fire Next Time         The Fire Next Time          100   

                                          Categories  More_Than_one_merge  
0  ['Economics', 'Nonfiction', 'Business', 'Polit...                    0  
1  ['Fiction', 'Classics', 'Science Fiction', 'Ph...                    0  
2  ['Fantasy', 'Fiction', 'Science Fiction', 'Cla...                    0  
3  ['Fiction', 'Short Stories', 'Classics', 'Fran...                    0  
4  ['Nonfiction', 'Essays', 'Classics', 'Race', '...                    0  
88
                     Title                       Book  Ratio_Value  \
0  Economics in one lesson    Economics in One 

100
                     Title                       Book  Ratio_Value  \
0  Economics in one lesson    Economics in One Lesson           91   
1                   Island                     Island          100   
2                   Island                   Islandia           86   
3    Man Who Planted Trees  The Man Who Planted Trees           91   
4       The Fire Next Time         The Fire Next Time          100   

                                          Categories  More_Than_one_merge  
0  ['Economics', 'Nonfiction', 'Business', 'Polit...                    0  
1  ['Fiction', 'Classics', 'Science Fiction', 'Ph...                    0  
2  ['Fantasy', 'Fiction', 'Science Fiction', 'Cla...                    0  
3  ['Fiction', 'Short Stories', 'Classics', 'Fran...                    0  
4  ['Nonfiction', 'Essays', 'Classics', 'Race', '...                    0  
85
                     Title                       Book  Ratio_Value  \
0  Economics in one lesson    Economics in One

94
                     Title                       Book  Ratio_Value  \
0  Economics in one lesson    Economics in One Lesson           91   
1                   Island                     Island          100   
2                   Island                   Islandia           86   
3    Man Who Planted Trees  The Man Who Planted Trees           91   
4       The Fire Next Time         The Fire Next Time          100   

                                          Categories  More_Than_one_merge  
0  ['Economics', 'Nonfiction', 'Business', 'Polit...                    0  
1  ['Fiction', 'Classics', 'Science Fiction', 'Ph...                    0  
2  ['Fantasy', 'Fiction', 'Science Fiction', 'Cla...                    0  
3  ['Fiction', 'Short Stories', 'Classics', 'Fran...                    0  
4  ['Nonfiction', 'Essays', 'Classics', 'Race', '...                    0  
86
                     Title                       Book  Ratio_Value  \
0  Economics in one lesson    Economics in One 

KeyboardInterrupt: 